In [1]:
from cobra.io import load_json_model
from cobra.flux_analysis import pfba
from os import getcwd
import math
import numpy as np
import pandas as pd
import holoviews as hv
from __future__ import unicode_literals
import matplotlib

In [2]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite, Model

In [3]:
matplotlib.interactive(True)

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
relative_directory = getcwd()
mcaps = load_json_model(relative_directory + '/Reconstructions/MethylococcusModel15.json')

## Reactions using Urea

In [6]:
mcaps.metabolites.urea_p.reactions

frozenset({<Reaction UREAabcpp at 0x10bec2190>,
           <Reaction UREAtex at 0x10bec2210>})

In [7]:
mcaps.reactions.DADK.gene_reaction_rule

u'MCA2066'

In [8]:
for rxn in mcaps.metabolites.urea_c.reactions:
    print rxn.id, rxn.gene_reaction_rule

BLUB MCA2289
UREASE_GAPFILLING 
UREAabcpp MCA1662


In [9]:
mcaps.medium

{u'EX_ca2_e': 1000,
 u'EX_cd2_e': 1000,
 u'EX_ch4_e': 18.46,
 u'EX_cl_e': 1000,
 u'EX_co_e': 1000,
 u'EX_cobalt2_e': 1000,
 u'EX_cu2_e': 1000,
 u'EX_fe2_e': 1000,
 u'EX_h2o_e': 1000,
 u'EX_h_e': 1000,
 u'EX_k_e': 1000,
 u'EX_mg2_e': 1000,
 u'EX_mn2_e': 1000,
 u'EX_n2_e': 1000,
 u'EX_na1_e': 1000,
 u'EX_ni2_e': 1000,
 u'EX_o2_e': 1000,
 u'EX_pi_e': 1000,
 u'EX_so4_e': 1000,
 u'EX_zn2_e': 1000}

## Results Collection

In [10]:
df = pd.DataFrame(index=["Methane UTR","Oxygen UTR", "CO2 UTR", "NH4 UTR", "N2 UTR",
                         "NO3 UTR", "Uphill Electron Transfer Flux", "Growth Rate" ,
                         "Growth Yield", "O2/CH4 Ratio", "NH4/CH4 Ratio", "N2/CH4 Ratio",
                         "NO3/CH4 Ratio", "ATP/CH4 Ratio"])

In [11]:
def df_record(solution, df, column):
    gr = solution.fluxes.BIOMASS_Mcapsulatus
    yx = gr/(abs((solution.fluxes.EX_ch4_e/1000)*16.04))
    data = [solution.fluxes.EX_ch4_e, solution.fluxes.EX_o2_e, solution.fluxes.EX_co2_e, solution.fluxes.EX_nh4_e,
            solution.fluxes.EX_n2_e, solution.fluxes.EX_no3_e, solution.fluxes.CYOR_q8ppi, gr, yx, 
            solution.fluxes.EX_o2_e/solution.fluxes.EX_ch4_e, solution.fluxes.EX_nh4_e/solution.fluxes.EX_ch4_e,
            solution.fluxes.EX_n2_e/solution.fluxes.EX_ch4_e, solution.fluxes.EX_no3_e/solution.fluxes.EX_ch4_e,
            solution.fluxes.ATPSipp/abs(solution.fluxes.EX_ch4_e)]
    df[column] = data
    return df

In [12]:
mcaps.reactions.EX_n2_e.bounds = 0, 1000
mcaps.reactions.ATPSNaipp.bounds = 0, 0
mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000

In [13]:
def highlevel_stoichiometry(solution):
    boundary_rxns = set(mcaps.exchanges) & set(solution.reactions)
    for rxn in boundary_rxns:
        if rxn.flux < 0:
            print([met.name for met in rxn.metabolites], round(rxn.flux, 5))
    for rxn in boundary_rxns:
        if rxn.flux > 0:
            print([met.name for met in rxn.metabolites], round(rxn.flux, 5))

## Redox-Arm Only

Carbon-Limited, NO3, pMMO, Redox-Arm Only

In [14]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, Redox-Arm Only')

In [15]:
highlevel_stoichiometry(solution)

([u'fe2_p'], -0.00109)
([u'no3_e'], -1.45857)
([u'dioxygen'], -27.8071)
([u'methane'], -18.46)
([u'Cl(-)'], -0.03817)
([u'Ca(2+)'], -0.01244)
([u'sulfate'], -0.02999)
([u'Nickel'], -1e-05)
([u'Cu2+'], -0.00027)
([u'Co(2+)'], -1e-05)
([u'Zinc'], -0.00018)
([u'H(+)'], -1.45807)
([u'phosphate'], -0.09799)
([u'K(+)'], -0.03142)
([u'Mg(2+)'], -0.02198)
([u'Manganese(2+)'], -0.0)
([u'Na(+)'], -0.00697)
([u'hydrogen peroxide'], 0.05725)
([u'carbon monoxide(1+)'], 1e-05)
([u'CO(2)'], 12.46802)
([u'H2O'], 32.96016)
([u'4-Hydroxy-benzyl alcohol'], 1e-05)
([u'doxopa_c'], 0.00019)
([u'alpha-ribazole'], 0.00019)


Carbon-Limited, NO3, sMMO, Redox-Arm Only

In [16]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, sMMO, Redox-Arm Only')

Carbon-Limited, NH3, pMMO, Redox-Arm Only

In [17]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, Redox-Arm Only')

In [18]:
highlevel_stoichiometry(solution)

([u'fe2_p'], -0.00159)
([u'dioxygen'], -27.86102)
([u'Ammonium'], -2.13275)
([u'methane'], -18.46)
([u'Cl(-)'], -0.0558)
([u'Ca(2+)'], -0.01818)
([u'sulfate'], -0.04393)
([u'Nickel'], -1e-05)
([u'Cu2+'], -0.00039)
([u'Co(2+)'], -2e-05)
([u'Zinc'], -0.00026)
([u'phosphate'], -0.1435)
([u'K(+)'], -0.04593)
([u'Mg(2+)'], -0.03213)
([u'Manganese(2+)'], -0.0)
([u'Na(+)'], -0.01019)
([u'hydrogen peroxide'], 0.08361)
([u'carbon monoxide(1+)'], 1e-05)
([u'CO(2)'], 9.69903)
([u'H2O'], 33.26359)
([u'H(+)'], 2.13306)
([u'4-Hydroxy-benzyl alcohol'], 1e-05)
([u'doxopa_c'], 0.00028)
([u'alpha-ribazole'], 0.00028)


Carbon-Limited, NH3, sMMO, Redox-Arm Only

In [19]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, sMMO, Redox-Arm Only')

## Redox-Arm Parameter Exploration

In [20]:
mcaps.reactions.ATPSipp.metabolites

{<Metabolite adp_c at 0x10baaac50>: -1.0,
 <Metabolite atp_c at 0x10bab4910>: 1.0,
 <Metabolite h2o_c at 0x10bada950>: 1.0,
 <Metabolite h_c at 0x10badaa90>: 3.0,
 <Metabolite h_p at 0x10badab10>: -4.0,
 <Metabolite pi_c at 0x10baf9b90>: -1.0}

In [21]:
for r in np.arange(0,2,0.2):
    with mcaps:
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 0
        mcaps.reactions.ATPSipp.add_metabolites(
            {mcaps.metabolites.adp_c: -1.0, 
             mcaps.metabolites.atp_c: 1.0,
             mcaps.metabolites.h2o_c: 1.0,
             mcaps.metabolites.h_c: (3.0-r),
             mcaps.metabolites.h_p: -(4.0-r), 
             mcaps.metabolites.pi_c: -1.0}, combine=False)
        solution = pfba(mcaps)
#         print(mcaps.reactions.ATPSipp)
#         print(solution.fluxes.ATPSipp)

    df = df_record(solution, df, 'NO3, pMMO, Redox-Arm, H/ATP {}'.format(3-r))

## Direct Coupling Exploration

Carbon-Limited, NO3, pMMODC

In [22]:
for r in np.arange(0,2.5,0.5):
    with mcaps:
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.PMMOipp.bounds = 0, 1000
        mcaps.reactions.PMMODCipp.bounds = 0, 1000
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 0
        c3 = mcaps.problem.Constraint(
                mcaps.reactions.PMMOipp.flux_expression - 
                r* mcaps.reactions.PMMODCipp.flux_expression, lb=0, ub=0, name='c3')
        mcaps.add_cons_vars(c3)
        solution = pfba(mcaps)
        print(solution.fluxes.MEOHDHipp,solution.fluxes.PMMODCipp, solution.fluxes.PMMOipp)

    df = df_record(solution, df, 'NO3, pMMODC, RedEff {}'.format(r))

(18.46, 18.46, 0.0)
(18.46, 12.306666666666668, 6.153333333333333)
(18.45999999999992, 9.22999999999996, 9.22999999999996)
(18.46, 7.384000000000001, 11.076000000000002)
(18.46, 6.153333333333333, 12.306666666666667)


## Uphill ET Parameter Exploration

Carbon-Limited, NO3, pMMO, Range (-5 - 0) 

In [13]:
for r in np.arange(-5,0,0.2):
    with mcaps:
        mcaps.reactions.CYOR_q8ppi.bounds = r, 1000
        mcaps.reactions.EX_no3_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 1000
        mcaps.reactions.ALAD_L.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NO3, pMMO, Uphill ET, {}'.format(r))

Carbon-Limited, NH3, pMMO, Range (-5 - 0)

In [14]:
for r in np.arange(-5,0,0.2):
    with mcaps:
        mcaps.reactions.CYOR_q8ppi.bounds = r, 1000
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.GLNS.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NH3, pMMO, Uphill ET, {}'.format(r))

Carbon-Limited, NO3, pMMO, Uphill Unconstrained

In [15]:
with mcaps:
    mcaps.reactions.CYOR_q8ppi.bounds = -1000, 1000
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, Uphill Unconstrained')

Carbon-Limited, NH4, pMMO, Uphill Unconstrained

In [16]:
with mcaps:
    mcaps.reactions.CYOR_q8ppi.bounds = -1000, 1000
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, Uphill Unconstrained')

In [17]:
df

,"NO3, pMMO, Redox-Arm Only","NO3, sMMO, Redox-Arm Only","NH3, pMMO, Redox-Arm Only","NH3, sMMO, Redox-Arm Only","NO3, pMMO, Uphill ET, -5.0","NO3, pMMO, Uphill ET, -4.8","NO3, pMMO, Uphill ET, -4.6","NO3, pMMO, Uphill ET, -4.4","NO3, pMMO, Uphill ET, -4.2","NO3, pMMO, Uphill ET, -4.0","NO3, pMMO, Uphill ET, -3.8","NO3, pMMO, Uphill ET, -3.6","NO3, pMMO, Uphill ET, -3.4","NO3, pMMO, Uphill ET, -3.2","NO3, pMMO, Uphill ET, -3.0","NO3, pMMO, Uphill ET, -2.8","NO3, pMMO, Uphill ET, -2.6","NO3, pMMO, Uphill ET, -2.4","NO3, pMMO, Uphill ET, -2.2","NO3, pMMO, Uphill ET, -2.0","NO3, pMMO, Uphill ET, -1.8","NO3, pMMO, Uphill ET, -1.6","NO3, pMMO, Uphill ET, -1.4","NO3, pMMO, Uphill ET, -1.2","NO3, pMMO, Uphill ET, -1.0","NO3, pMMO, Uphill ET, -0.8","NO3, pMMO, Uphill ET, -0.6","NO3, pMMO, Uphill ET, -0.4","NO3, pMMO, Uphill ET, -0.2","NH3, pMMO, Uphill ET, -5.0","NH3, pMMO, Uphill ET, -4.8","NH3, pMMO, Uphill ET, -4.6","NH3, pMMO, Uphill ET, -4.4","NH3, pMMO, Uphill ET, -4.2","NH3, pMMO, Uphill ET, -4.0","NH3, pMMO, Uphill ET, -3.8","NH3, pMMO, Uphill ET, -3.6","NH3, pMMO, Uphill ET, -3.4","NH3, pMMO, Uphill ET, -3.2","NH3, pMMO, Uphill ET, -3.0","NH3, pMMO, Uphill ET, -2.8","NH3, pMMO, Uphill ET, -2.6","NH3, pMMO, Uphill ET, -2.4","NH3, pMMO, Uphill ET, -2.2","NH3, pMMO, Uphill ET, -2.0","NH3, pMMO, Uphill ET, -1.8","NH3, pMMO, Uphill ET, -1.6","NH3, pMMO, Uphill ET, -1.4","NH3, pMMO, Uphill ET, -1.2","NH3, pMMO, Uphill ET, -1.0","NH3, pMMO, Uphill ET, -0.8","NH3, pMMO, Uphill ET, -0.6","NH3, pMMO, Uphill ET, -0.4","NH3, pMMO, Uphill ET, -0.2","NO3, pMMO, Uphill Unconstrained","NH3, pMMO, Uphill Unconstrained"
Methane UTR,-18.460000,-18.460000,-18.460000,-18.460000,-1.846000e+01,-1.846000e+01,-1.846000e+01,-1.846000e+01,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-18.460000,-1.846000e+01
Oxygen UTR,-27.807105,-27.806966,-27.861020,-27.861020,-2.533865e+01,-2.543738e+01,-2.553611e+01,-2.563485e+01,-25.733578,-25.832311,-25.931043,-26.029776,-26.128509,-26.227242,-26.325974,-26.424707,-26.523440,-26.622173,-26.720905,-26.819638,-26.918371,-27.017104,-27.115837,-27.214569,-27.313302,-27.412035,-27.510768,-27.609500,-27.708233,-25.407342,-25.505489,-25.603636,-25.701783,-25.799930,-25.898077,-25.996224,-26.094372,-26.192519,-26.290666,-26.388813,-26.486960,-26.585107,-26.683254,-26.781401,-26.879549,-26.977696,-27.075843,-27.173990,-27.272137,-27.370284,-27.468431,-27.566578,-27.664726,-27.762873,-23.195927,-2.401151e+01
CO2 UTR,12.468019,12.468098,9.699031,9.699031,1.084516e+01,1.091007e+01,1.097499e+01,1.103991e+01,11.104827,11.169745,11.234662,11.299580,11.364498,11.429415,11.494333,11.559251,11.624168,11.689086,11.754004,11.818921,11.883839,11.948757,12.013674,12.078592,12.143510,12.208427,12.273345,12.338263,12.403181,7.326070,7.420989,7.515907,7.610826,7.705744,7.800662,7.895581,7.990499,8.085418,8.180336,8.275254,8.370173,8.465091,8.560010,8.654928,8.749847,8.844765,8.939683,9.034602,9.129520,9.224439,9.319357,9.414275,9.509194,9.604112,9.436299,5.976158e+00
NH4 UTR,0.000000,0.000000,-2.132753,-2.132753,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.710422,-2.687315,-2.664208,-2.641102,-2.617995,-2.594888,-2.571781,-2.548675,-2.525568,-2.502461,-2.479354,-2.456248,-2.433141,-2.410034,-2.386927,-2.363821,-2.340714,-2.317607,-2.294500,-2.271394,-2.248287,-2.225180,-2.202073,-2.178967,-2

## 'Turning on' the uphill electron transfer

Using the value that was fit above

In [18]:
mcaps.reactions.CYOR_q8ppi.bounds = -2.8, 1000

Carbon-Limited, NO3, sMMO

In [19]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, sMMO')

Carbon-Limited, NO3, pMMO

In [20]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO')

Carbon-Limited, NH3, sMMO

In [21]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, sMMO')

Carbon-Limited, NH3, pMMO

In [22]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO')

Carbon-Limited, N2, sMMO

In [23]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, sMMO')

Carbon-Limited, N2, pMMO

In [24]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, pMMO')

## Direct Coupling 

Carbon-Limited, NO3, pMMODC

In [25]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMODC')

Carbon-Limited, NH3, pMMODC

In [26]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH4, pMMODC')

Carbon-Limited, N2, pMMODC

In [27]:
with mcaps:
    mcaps.reactions.EX_n2_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 0
    mcaps.reactions.PMMODCipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'N2, pMMODC')

## Effect of NH3 oxidation

Carbon-Limited, NH3, pMMO, NO2

In [28]:
with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    c3 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
    mcaps.add_cons_vars(c3)
    mcaps.reactions.GLNS.bounds = 0, 0
    solution = pfba(mcaps)

df = df_record(solution, df, 'NH3, pMMO, 1/2*NO2')

Carbon-Limited, NH3, pMMO, NO2, Range(0-5)

In [29]:
np.arange(1,4.2,0.2)

array([1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4, 2.6, 2.8, 3. , 3.2, 3.4,
       3.6, 3.8, 4. ])

In [30]:
for r in np.arange(1,4.2,0.2):
    with mcaps:
        mcaps.reactions.EX_nh4_e.bounds = -1000, 0
        mcaps.reactions.PMMOipp.bounds = 0, 1000
        c3 = mcaps.problem.Constraint(
        mcaps.reactions.EX_nh4_e.flux_expression + r* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
        mcaps.add_cons_vars(c3)
        mcaps.reactions.GLNS.bounds = 0, 0
        solution = pfba(mcaps)

    df = df_record(solution, df, 'NH3, pMMO, 1/{}*NO2'.format(r))

Carbon-Limited, NO3, pMMO, NO2

In [31]:
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    c3 = mcaps.problem.Constraint(
    mcaps.reactions.EX_nh4_e.flux_expression + 2* mcaps.reactions.EX_no2_e.flux_expression, lb=0, ub=0, name='c3')
    mcaps.add_cons_vars(c3)
    mcaps.reactions.GLNS.bounds = 0, 1000
    solution = pfba(mcaps)

df = df_record(solution, df, 'NO3, pMMO, 1/2*NO2')

O2-Limited, NH3, pMMO, 0.29

In [32]:
with mcaps:
    mcaps.objective = mcaps.reactions.EX_o2_e
    mcaps.reactions.EX_nh4_e.bounds = -1000, 0
    mcaps.reactions.PMMOipp.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 0
    mcaps.reactions.BIOMASS_Mcapsulatus.bounds = 0.291, 0.291
    solution = pfba(mcaps)

df = df_record(solution, df, 'O2-Limited, NH3, pMMO, 0.29')

C-Limited, NO3, pMMO, KOTerminalOxidase

In [33]:
# mcaps.reactions.CYTAA3ipp

In [34]:
# with mcaps:
#     mcaps.reactions.EX_nh4_e.bounds = -1000, 0
#     mcaps.reactions.PMMOipp.bounds = 0, 1000
#     mcaps.reactions.GLNS.bounds = 0, 1000
#     mcaps.reactions.CYTAA3ipp.bounds = 0, 0
#     mcaps.reactions.CYTCBDipp.bounds = 0, 0
#     solution = pfba(mcaps)

# df = df_record(solution, df, 'NO3, pMMO, KOTerminalOxidase')

In [35]:
df.round(2)

,"NO3, pMMO, Redox-Arm Only","NO3, sMMO, Redox-Arm Only","NH3, pMMO, Redox-Arm Only","NH3, sMMO, Redox-Arm Only","NO3, pMMO, Uphill ET, -5.0","NO3, pMMO, Uphill ET, -4.8","NO3, pMMO, Uphill ET, -4.6","NO3, pMMO, Uphill ET, -4.4","NO3, pMMO, Uphill ET, -4.2","NO3, pMMO, Uphill ET, -4.0","NO3, pMMO, Uphill ET, -3.8","NO3, pMMO, Uphill ET, -3.6","NO3, pMMO, Uphill ET, -3.4","NO3, pMMO, Uphill ET, -3.2","NO3, pMMO, Uphill ET, -3.0","NO3, pMMO, Uphill ET, -2.8","NO3, pMMO, Uphill ET, -2.6","NO3, pMMO, Uphill ET, -2.4","NO3, pMMO, Uphill ET, -2.2","NO3, pMMO, Uphill ET, -2.0","NO3, pMMO, Uphill ET, -1.8","NO3, pMMO, Uphill ET, -1.6","NO3, pMMO, Uphill ET, -1.4","NO3, pMMO, Uphill ET, -1.2","NO3, pMMO, Uphill ET, -1.0","NO3, pMMO, Uphill ET, -0.8","NO3, pMMO, Uphill ET, -0.6","NO3, pMMO, Uphill ET, -0.4","NO3, pMMO, Uphill ET, -0.2","NH3, pMMO, Uphill ET, -5.0","NH3, pMMO, Uphill ET, -4.8","NH3, pMMO, Uphill ET, -4.6","NH3, pMMO, Uphill ET, -4.4","NH3, pMMO, Uphill ET, -4.2","NH3, pMMO, Uphill ET, -4.0","NH3, pMMO, Uphill ET, -3.8","NH3, pMMO, Uphill ET, -3.6","NH3, pMMO, Uphill ET, -3.4","NH3, pMMO, Uphill ET, -3.2","NH3, pMMO, Uphill ET, -3.0","NH3, pMMO, Uphill ET, -2.8","NH3, pMMO, Uphill ET, -2.6","NH3, pMMO, Uphill ET, -2.4","NH3, pMMO, Uphill ET, -2.2","NH3, pMMO, Uphill ET, -2.0","NH3, pMMO, Uphill ET, -1.8","NH3, pMMO, Uphill ET, -1.6","NH3, pMMO, Uphill ET, -1.4","NH3, pMMO, Uphill ET, -1.2","NH3, pMMO, Uphill ET, -1.0","NH3, pMMO, Uphill ET, -0.8","NH3, pMMO, Uphill ET, -0.6","NH3, pMMO, Uphill ET, -0.4","NH3, pMMO, Uphill ET, -0.2","NO3, pMMO, Uphill Unconstrained","NH3, pMMO, Uphill Unconstrained","NO3, sMMO","NO3, pMMO","NH3, sMMO","NH3, pMMO","N2, sMMO","N2, pMMO","NO3, pMMODC","NH4, pMMODC","N2, pMMODC","NH3, pMMO, 1/2*NO2","NH3, pMMO, 1/1.0*NO2","NH3, pMMO, 1/1.2*NO2","NH3, pMMO, 1/1.4*NO2","NH3, pMMO, 1/1.6*NO2","NH3, pMMO, 1/1.8*NO2","NH3, pMMO, 1/2.0*NO2","NH3, pMMO, 1/2.2*NO2","NH3, pMMO, 1/2.4*NO2","NH3, pMMO, 1/2.6*NO2","NH3, pMMO, 1/2.8*NO2","NH3, pMMO, 1/3.0*NO2","NH3, pMMO, 1/3.2*NO2","NH3, pMMO, 1/3.4*NO2","NH3, pMMO, 1/3.6*NO2","NH3, pMMO, 1/3.8*NO2","NH3, pMMO, 1/4.0*NO2","NO3, pMMO, 1/2*NO2","O2-Limited, NH3, pMMO, 0.29"
Methane UTR,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-2.58,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-18.46,-17.84
Oxygen UTR,-27.81,-27.81,-27.86,-27.86,-25.34,-25.44,-25.54,-25.63,-25.73,-25.83,-25.93,-26.03,-26.13,-26.23,-26.33,-26.42,-26.52,-26.62,-26.72,-26.82,-26.92,-27.02,-27.12,-27.21,-27.31,-27.41,-27.51,-27.61,-27.71,-25.41,-25.51,-25.60,-25.70,-25.80,-25.90,-26.00,-26.09,-26.19,-26.29,-26.39,-26.49,-26.59,-26.68,-26.78,-26.88,-26.98,-27.08,-27.17,-27.27,-27.37,-27.47,-27.57,-27.66,-27.76,-23.20,-24.01,-27.03,-26.42,-27.86,-26.49,-28.11,-26.77,-21.32,-22.25,-23.95,-29.77,-5.16,-38.09,-33.59,-31.60,-30.49,-29.77,-29.27,-28.90,-28.62,-28.40,-28.22,-28.07,-27.94,-27.84,-27.74,-27.66,-26.42,-25.55
CO2 UTR,12.47,12.47,9.70,9.70,10.85,10.91,10.97,11.04,11.10,11.17,11.23,11.30,11.36,11.43,11.49,11.56,11.62,11.69,11.75,11.82,11.88,11.95,12.01,12.08,12.14,12.21,12.27,12.34,12.40,7.33,7.42,7.52,7.61,7.71,7.80,7.90,7.99,8.09,8.18,8.28,8.37,8.47,8.56,8.65,8.75,8.84,8.94,9.03,9.13,9.22,9.32,9.41,9.51,9.60,9.44,5.98,11.95,11.56,9.70,8.37,10.58,9.39,8.20,4.27,6.86,9.42,2.58,12.06,10.63,10.00,9.64,9.42,9.26,9.14,9.05,8.98,8.92,8.87,8.83,8.80,8.77,8.74,11.56,8.04
NH4 UTR,0.00,0.00,-2.13,-2.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-2.7

In [65]:
df.to_csv("/Users/clie/Desktop/EFPro2/MethylococcusCapsulatusValidation.csv")

In [36]:
import matplotlib.pyplot as plt

In [42]:
plt.plot([1,2,3,4])

In [43]:
plt.draw()

In [44]:
plt.show()

In [31]:
import matplotlib.pyplot as plt

In [32]:
plt.figure(figsize=(5,3), dpi=300);

df[['NO3, sMMO','NO3, pMMO','NH3, sMMO','NH3, pMMO','N2, sMMO','N2, pMMO']].round(2).loc['Growth Yield'].plot.bar(); plt.axhline(0, color='k')

In [ ]:
df[['NO3, sMMO','NO3, pMMO','NH3, sMMO','NH3, pMMO','N2, sMMO','N2, pMMO']]

In [ ]:
plt.figure(figsize=(5,3), dpi=300);

df[['NO3, sMMO','NO3, pMMO','NH3, sMMO','NH3, pMMO','N2, sMMO','N2, pMMO']].round(2).loc['O2/CH4 Ratio'].plot.bar(ylim=(1.1,1.7)); plt.axhline(0, color='k')

In [ ]:
plt.figure(figsize=(5,3), dpi=300);

df[['NO3, pMMODC','NH4, pMMODC','N2, pMMODC','NH3, pMMO, 1/2*NO2']].loc['O2/CH4 Ratio'].plot.bar(ylim=(1.1,1.7))